# LangChain 1.0: Agents, HITL, and Multi-Agent Systems

Welcome to this guide on leveraging LangChain 1.0! In this notebook, we'll explore the modern way to build agents using the `create_agent` workflow.

We will cover:
1. **Simple Agents**: Building a basic agent with tools.
2. **Human-in-the-Loop (HITL)**: Adding oversight to our agents.
3. **Retrieval**: Giving our agent access to external data (RAG) using OpenAI embeddings and Qdrant.
4. **Multi-Agent Systems**: Composing multiple agents together.

All the core logic is mirrored in the `src/` directory, demonstrating how to structure your project for deployment with LangSmith.


## 1. Setup and Installation
Let's get our environment ready.

In [ ]:
!pip install -qU langchain langchain-openai langchain-community langchain-qdrant qdrant-client pymupdf langgraph langsmith beautifulsoup4

## 2. Environment Variables
We need to set our OpenAI API Key. We also enable LangSmith tracing here.

In [ ]:
import os
import getpass

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")

# LangSmith Tracing
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "LangChain 1.0 Guide"
_set_if_undefined("LANGSMITH_API_KEY")


## 3. The `create_agent` Workflow

LangChain 1.0 introduces `create_agent` as the standard way to build agents. It simplifies the process while retaining the power of LangGraph under the hood.

We've defined a simple agent in `src/agent.py` that has access to a weather tool and a "magic calculator".


In [ ]:
from src.agent import build_simple_agent

# Build the agent
agent = build_simple_agent()

# Run the agent
response = agent.invoke({"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]})
print(response["messages"][-1].content)


## 4. Human-in-the-Loop (HITL)

Sometimes we want to approve sensitive actions before they happen. LangChain 1.0 makes this easy with middleware.

We've configured our `hitl_agent` to interrupt before using the `magic_calculator` tool.


In [ ]:
from src.agent import build_hitl_agent
from langgraph.types import Command
# Import uuid7 for generating valid thread IDs compliant with LangSmith
from langsmith import uuid7

# Build the HITL agent
hitl_agent = build_hitl_agent()

# Configuration for the thread (required for checkpointing)
# Using uuid7 to avoid warnings and ensure best practice
thread_id = str(uuid7())
config = {"configurable": {"thread_id": thread_id}}

print(f"--- Asking to calculate (Thread: {thread_id}) ---")
# We use .stream() to see steps
events = list(hitl_agent.stream(
    {"messages": [{"role": "user", "content": "Please use the magic calculator to add 5 and 5."}]},
    config=config
))

# Print events to verify the initial run and interruption
for i, event in enumerate(events):
    if "messages" in event:
        event["messages"][-1].pretty_print()


You should see the tool call request, and then the stream ends (due to interruption). We can now resume.

In [ ]:
# Resume execution
# We need to construct the resume payload dynamically based on the interrupt ID.

# 1. Fetch the current state
state = hitl_agent.get_state(config)

# 2. Find the interrupt
tasks = state.tasks
resume_payload = {}

if tasks and tasks[0].interrupts:
    interrupt = tasks[0].interrupts[0]
    print(f"Found interrupt: {interrupt.id}")
    
    # 3. Construct payload mapping interrupt ID to decision
    resume_payload = {
        interrupt.id: {
            "decisions": [{"type": "approve"}]
        }
    }
else:
    print("No active interrupts found. Agent might have finished.")

# 4. Resume if we have a payload
if resume_payload:
    print(f"--- Resuming (Thread: {thread_id}) ---")
    resume_output = list(hitl_agent.stream(
        Command(resume=resume_payload),
        config=config
    ))
    
    if not resume_output:
        print("No events received after resume. Checking final state...")
        final_state = hitl_agent.get_state(config)
        if final_state.values and "messages" in final_state.values:
             final_state.values["messages"][-1].pretty_print()
    else:
        for event in resume_output:
            if "messages" in event:
                event["messages"][-1].pretty_print()
            else:
                print(event)


## 5. Retrieval (RAG)

Agents become truly powerful when they can access external knowledge. We can add a retriever as a tool.

We've set up a RAG pipeline to search the official LangChain blog for updates about the **LangChain 1.0 release**.


In [ ]:
from src.agent import build_rag_agent

rag_agent = build_rag_agent()

response = rag_agent.invoke({
    "messages": [{"role": "user", "content": "When should I use LangChain vs LangGraph?"}]
})

print(response["messages"][-1].content)


## 6. Multi-Agent Systems

LangChain 1.0 agents are graphs, which means they can be composed! We can treat an agent as a tool for another agent.

We have a "Supervisor" that delegates to a "Researcher" (who has RAG access) and a "Writer" (who has a specific persona).


In [ ]:
from src.agent import build_multi_agent_system

supervisor = build_multi_agent_system()

# This complex query requires research and then writing
query = "Research the key features of LangChain 1.0 and write a short poem about them."

response = supervisor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print(response["messages"][-1].content)
